## Importing Packages

In [1]:
#Import Needed Packages

import numpy as np
import pandas as pd
import os
import math
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
from tqdm import tqdm
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import KFold
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from tqdm import tqdm
from wordcloud import WordCloud,STOPWORDS
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.preprocessing import StandardScaler

nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import initializers, regularizers

from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,mean_squared_log_error

from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from scipy.stats import randint
from sklearn.model_selection import  RandomizedSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Msys\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Msys\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Msys\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##  Loading Data

In [2]:
train = pd.read_csv(r"C:\Users\Msys\Desktop\Data Science-GSG\train.tsv",sep='\t')

In [3]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   train_id           1482535 non-null  int64  
 1   name               1482535 non-null  object 
 2   item_condition_id  1482535 non-null  int64  
 3   category_name      1476208 non-null  object 
 4   brand_name         849853 non-null   object 
 5   price              1482535 non-null  float64
 6   shipping           1482535 non-null  int64  
 7   item_description   1482531 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 90.5+ MB


# Pipline Functions

### Fill Missing values

In [5]:
def imputing(data):
    def mode_func(x):
        return x.mode().iloc[0] if not x.mode().empty else None
    # grouped the data by (category_name) then we got the brand_name mode of each group
    brand_name_grouped = data.groupby('category_name')['brand_name']
    grouped = brand_name_grouped.agg(mode_func).reset_index()

    #fill the missing value in each group with the brand_name mode of its group 
    modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    # Fill missing values in 'brand_name' with the calculated modes within each group
    data['brand_name'] = data['brand_name'].fillna(modes)
    return data

In [6]:
def fill_missing_data(data):
    data.brand_name.fillna(value="No Brand", inplace=True)
    data.category_name.fillna(value = "No Category", inplace = True)
    data.item_description.fillna(value = "No description yet", inplace = True)
    return data

### Drop Item with Price 0

In [10]:
def drop_price_zero(data):
    data = data[data['price'] != 0]
    return data

### Split  category_name into  :  main_category,  category_sub1,  category_sub2

In [12]:
def category_tranformation(category):
    try:
        category, sub_category_1, sub_category_2 = category.split('/')
        return category, sub_category_1, sub_category_2
    except:
        return 'Other', 'Other', 'Other'

def sub_categories(df):
    df['main_category'], df['category_sub1'], df['category_sub2'] = zip(*df['category_name'].apply(category_tranformation))
    train_category = df[['main_category','category_sub1','category_sub2']]
    
    return df

# Encoding

### One Hot Encoding 

In [14]:
# Define the one-hot encoding for 'main_category', 'category_sub1', 'category_sub2','item_condition_id'
def one_hot_encoding(df):
     # List of columns to one-hot encode
    columns_to_encode = ['item_condition_id','main_category', 'category_sub1', 'category_sub2']
    # Perform one-hot encoding using pd.get_dummies
    df_encoded = pd.get_dummies(df, columns=columns_to_encode)
    #df = pd.concat((df, pd.get_dummies(data=df.item_condition_id, prefix='item_condition_id')), axis=1)
    return df_encoded

### Label Encoding 

In [15]:
#  Define Label encoding for 'name','brand_name'
def label_encoder(data):   
    brand_label = LabelEncoder().fit_transform(data['brand_name'])        
    name_label = LabelEncoder().fit_transform(data['name'])
    return data

In [16]:
#train = one_hot_encoding(train)
#train = label_encoder(train)

### Number of words in item_description and name

In [17]:
%%time
# get name and description lengths
def wordCount(text):
    try:
        if text == 'No description yet':
            return 0
        else:
            text = text.lower()
            words = [w for w in text.split(" ")]
            return len(words)
    except: 
        return 0

CPU times: total: 0 ns
Wall time: 0 ns


### Is_branded 

In [19]:
def branded(data):
    # assigns a value 1 if a product has brand_name else 0
    is_branded=[]
    for i in data['brand_name']:
        if i=='No Brand': is_branded.append(0) #if it is a Nan value i.e.. unknown brand make it as 0.
        else: is_branded.append(1)
    return is_branded

We know that a product with different brands vary with their price. 
A good brand will have a good price compared to the same product of different brand.
Therefore two similar products with different brands(known brand,no brand) can help us to know the price of the product.

### Sentiments Score
 A score ranging between 0–100 % that shows the polarity of the sentence i.e how much is a sentence positive, negative, and neutral. A score of 100% shows total positivity and vice versa. An overall score is given by the compound.

In [ ]:
sia = SentimentIntensityAnalyzer()
def senti_score(data):
    data['compound_sentiment_score'] = data['item_description'].apply(lambda x: sia.polarity_scores(x)['compound'])
    return data 

### ColumnTransformer
- Applied ColumnTransformer with transformers — Counvectorizer, TfidfVectorizer, and Normalizer to transform the text and numerical features into a single feature space.

- CountVectorizer and TfidfVectorizer are both text vectorization techniques commonly used in natural language processing (NLP) and machine learning for converting text data into numerical feature vectors

- **Example** :

    - Document 1: "I love programming."
    - Document 2: "Programming is fun." 
    - Document 3: "I enjoy coding."

- Using CountVectorizer: CountVectorizer will convert this corpus into a matrix of word counts. Here's what it would look like:

    - Vocabulary: ["I", "love", "programming", "is", "fun", "enjoy", "coding"] 
        - Document 1: [1, 1, 1, 0, 0, 0, 0] 
        - Document 2: [0, 0, 1, 1, 1, 0, 0] 
        - Document 3: [1, 0, 0, 0, 0, 1, 1]

- Using TfidfVectorizer: TfidfVectorizer takes into account not just word counts but also the importance of each word in the corpus. It calculates TF-IDF scores for each word in each document:

    - Vocabulary: ["I", "love", "programming", "is", "fun", "enjoy", "coding"] 
        - Document 1: [0.59, 0.81, 0.5, 0, 0, 0, 0] 
        - Document 2: [0, 0, 0.5, 0.81, 0.81, 0, 0] 
        - Document 3: [0.59, 0, 0, 0, 0, 0.81, 0.81]

def col_transform(data):
    name_vec = CountVectorizer(ngram_range=(1, 2), max_features=150000, 
                               stop_words=['why', 'x', 'w', 's', 'e', 'ty', 'wgf'],
                               token_pattern = r'(?u)\w+|[^\w\s]')

    desc_vec = TfidfVectorizer(ngram_range=(1, 3), max_features=150000, token_pattern = r'(?u)\w+|[^\w\s]')

    bname_vec = CountVectorizer( ngram_range = (1, 2))
    cat_vec = CountVectorizer( token_pattern='.+')
    #numeric_features = [data['item_condition_id'], data['shipping'] , data['description_length'] , data[],'compound_sentiment_score']
    
    numeric_features = [data['item_condition_id'], data['shipping'] , data['description_length'] , data['Is_branded'],train['name_length']]
    
    col_transformer = ColumnTransformer(transformers=[('numerical', Normalizer(), numeric_features),
                                                          (data['name'], name_vec, data['name']),
                                                          (data['brand_name'], bname_vec, data['brand_name']),
                                                          (data['item_description'], desc_vec, data['item_description']),
                                                          (data['main_category'], cat_vec, data['main_category']),
                                                          (data['category_sub1'], cat_vec, data['category_sub1']),
                                                          (data['category_sub2'], cat_vec, data['category_sub2'])])
    vectorizer = col_transformer.fit(data)
    train_vec = vectorizer.transform(data)
    return train_vec
    

### Drop Columns

In [21]:
def drop_cols(data):
    data = data.drop(['train_id','category_name','name','brand_name','item_description'],axis = 1)
    return data

In [23]:
def preprocess_data(train):
    train = imputing(train)
    train = fill_missing_data(train)
    train=drop_price_zero(train)
    train= sub_categories(train)
    train = one_hot_encoding(train)
    train = label_encoder(train)
    train['description_length'] = train['item_description'].apply(lambda x: wordCount(x))
    train['name_length'] = train['name'].apply(lambda x: wordCount(x))
    train['is_branded']=branded(train)
    train=drop_cols(train)
    return train

In [24]:
train=preprocess_data(train)

C:\Users\Msys\AppData\Local\Temp\ipykernel_12100\3451237.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
C:\Users\Msys\AppData\Local\Temp\ipykernel_12100\3451237.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
C:\Users\Msys\AppData\Local\Temp\ipykernel_12100\3451237.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
C:\Users\Msys\AppData\Local\Tem

C:\Users\Msys\AppData\Local\Temp\ipykernel_12100\3451237.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
C:\Users\Msys\AppData\Local\Temp\ipykernel_12100\3451237.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
C:\Users\Msys\AppData\Local\Temp\ipykernel_12100\3451237.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
C:\Users\Msys\AppData\Local\Tem

C:\Users\Msys\AppData\Local\Temp\ipykernel_12100\3451237.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
C:\Users\Msys\AppData\Local\Temp\ipykernel_12100\3451237.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
C:\Users\Msys\AppData\Local\Temp\ipykernel_12100\3451237.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
C:\Users\Msys\AppData\Local\Tem

C:\Users\Msys\AppData\Local\Temp\ipykernel_12100\3451237.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
C:\Users\Msys\AppData\Local\Temp\ipykernel_12100\3451237.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
C:\Users\Msys\AppData\Local\Temp\ipykernel_12100\3451237.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  modes = brand_name_grouped.transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
C:\Users\Msys\AppData\Local\Tem

###  Train and Test Split:

In [25]:
size = int(0.70 * len(train))
train2 = train.sample(n=size, random_state=42)

In [26]:
X_data = train2.drop(['price'],axis = 1)
y_data = train2['price']

In [27]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# Modeling

In [36]:
#pip install lightgbm

# SVR

In [42]:
# Apply PCA to reduce dimensionality
pca = PCA(n_components=10)  # Specify the number of components you want to keep
X_train_pca_svr = pca.fit_transform(X_train)
X_test_pca_svr = pca.transform(X_test)

In [43]:
# Create an SVR model
svr_model = SVR(kernel='linear', C=1.0, epsilon=0.2)  

In [ ]:
# Train the model on the PCA-transformed training data
svr_model.fit(X_train_pca_svr, y_train)

In [ ]:
# Make predictions on the PCA-transformed test data
y_pred_svr = svr_model.predict(X_test_pca_svr)

In [ ]:
# Evaluate the model
mse_svr = mean_squared_error(y_test, y_pred_svr)
rmse_svr = mse_svr ** 0.5
r2_svr = r2_score(y_test, y_pred_svr)

print(f"Mean Squared Error (mse_svr): {mse_svr}")
print(f"Root Mean Squared Error (rmse_svr): {rmse_svr}")
print(f"R-squared (r2_svr) Score: {r2_svr}")

## LGBMRegressor

In [40]:
# Standardize the features (optional but recommended before PCA)
scaler = StandardScaler()

In [41]:
X_train_scaled = scaler.fit_transform(X_train)

In [42]:
X_test_scaled = scaler.transform(X_test)

In [43]:
# Apply PCA to reduce dimensionality
n_components = 10  # Adjust the number of components as needed
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)

In [44]:
X_test_pca = pca.transform(X_test_scaled)

In [45]:
# Create and train the LGBMRegressor model
model = LGBMRegressor(random_state=42)
model.fit(X_train_pca, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.079991 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 592664, number of used features: 10
[LightGBM] [Info] Start training from score 26.726794


LGBMRegressor(random_state=42)

In [46]:
# Make predictions on the test set
y_pred = model.predict(X_test_pca)

In [47]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2) Score: {r2}")

Mean Squared Error (MSE): 1233.4591288747047
Root Mean Squared Error (RMSE): 35.12063679483481
R-squared (R2) Score: 0.1151303778988172


## Ridge Model

In [50]:
# Create a Ridge regression model
ridge_model = Ridge(alpha=1.0)  # You can adjust the alpha (regularization strength) and other hyperparameters

In [51]:
# Train the model on the training data
ridge_model.fit(X_train, y_train)

Ridge()

In [52]:
# Make predictions on the test data
y_pred_ridge = ridge_model.predict(X_test)

In [53]:
# Evaluate the model
ridge_mse = mean_squared_error(y_test, y_pred_ridge)
ridge_rmse = mse ** 0.5
ridge_r2 = r2_score(y_test, y_pred_ridge)

print(f"Mean Squared Error (ridge_mse): {ridge_mse}")
print(f"Root Mean Squared Error (ridge_rmse): {ridge_rmse}")
print(f"R-squared (ridge_r2) Score: {ridge_r2}")

Mean Squared Error (ridge_mse): 1133.06085484235
Root Mean Squared Error (ridge_rmse): 35.12063679483481
R-squared (ridge_r2) Score: 0.18715496365357154


# Lasso Regression

In [35]:
# Apply PCA to reduce dimensionality
pca = PCA(n_components=10)  # Specify the number of components you want to keep
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [36]:
# Create a Lasso Regression model
lasso_model = Lasso(alpha=1.0)  # You can adjust the alpha (regularization strength) based on your problem

In [37]:
# Train the model on the PCA-transformed training data
lasso_model.fit(X_train_pca, y_train)

Lasso()

In [38]:
# Make predictions on the PCA-transformed test data
y_pred_lasso = lasso_model.predict(X_test_pca)

In [40]:
# Evaluate the model
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
rmse_lasso = mse_lasso ** 0.5
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f"Mean Squared Error (mse_lasso): {mse_lasso}")
print(f"Root Mean Squared Error (rmse_lasso): {rmse_lasso}")
print(f"R-squared (r2_lasso) Score: {r2_lasso}")

Mean Squared Error (mse_lasso): 1422.5094822965116
Root Mean Squared Error (rmse_lasso): 37.716170037485405
R-squared (r2_lasso) Score: 0.014193523903032634
